# Klassifikation 

In diesem Notebook trainieren wir einen Classifier. Aus Beispielen lernt der Classifier, zu welcher Klasse ein Element gehört, und kann neue Elemente mit den gelernten Regeln einordnen. 

Wir lesen zunächst Dokumente aus 4 Wikipedia-Kategorien ein.

In [93]:
import SaveWiki

SaveWiki.downloadWikiCat('Wirtschaftswissenschaft','Economics')
SaveWiki.downloadWikiCat('Angewandte Informatik','Applied')

In [94]:
import codecs 

def readtext(dateiname):
    text = ''
    d = codecs.open(dateiname,'r','utf8')
    for zeile in d:
        text += str(zeile)
    d.close()

    return text

In [95]:
bsptext= readtext('Economics/Finanzökonomik.txt')
print(bsptext[:300])

Die Finanzökonomik (eng. financial economics), auch Finanzmarktökonomik oder Kapitalmarktforschung, ist ein Bereich der Wirtschaftswissenschaft, der die Verwendung und Verteilung von  Ressourcen auf Kapitalmärkten analysiert.Die Finanzökonomik betrachtet die Wechselbeziehung von Variablen wie Preise


In [96]:
import nltk
import re
from HanTa import HanoverTagger as ht
from collections import Counter

stopwords = nltk.corpus.stopwords.words('german')
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

def closed_class(pos):
    if pos[0] == '$':
        return True
    elif pos in ["APPR", "APPRART", "APPO", "APZR", "ART", "KOUI", "KOUS", "KON", "KOKOM", "PDS", "PDAT", "PIS", "PIAT", "PIDAT", "PPER", "PPOSS", "PPOSAT", "PRELS", "PRELAT", "PRF", "PWS", "PWAT", "PWAV", "PAV", "PTKZU", "PTKNEG", "VAFIN", "VAIMP", "VAINF", "VAPP", "VMFIN", "VMINF", "VMPP"]:
        return True
    
    return False

def features_from_text(text):
    wordcounts = Counter()
    tlen = 0
    
    satzliste =  nltk.sent_tokenize(text,language='german')
    for satz in satzliste:
        tokens =  nltk.word_tokenize(satz,language='german')
        tokens = [lemma for (word,lemma,pos) in tagger.tag_sent(tokens) if not closed_class(pos)]
        tokens = [t for t in tokens if t.lower() not in stopwords]
        tokens = [t for t in tokens if re.search('^\w+$',t)]
        tlen += len(tokens)
        wordcounts.update(tokens)

    return {w:wordcounts[w]/tlen for w in wordcounts}

Wir versuchen aus jede Klasse 50 Dokumente zu lesen, die nicht extrem kurz oder lang sind.

In [97]:
import glob

def read_data(directories):
    docs = []
    for directory in directories:
        dirsize = 0
        for file in glob.glob(directory+"/*.txt"):
            text = readtext(file)
            if len(text) > 500 and len(text) < 10000:
                docs.append((features_from_text(text),directory))
                dirsize += 1
            if dirsize >= 50:
                break
    return docs

data = read_data(['Economics','Applied'])

Wir mischen die Daten und teilen in Test- und Trainingsdaten

In [98]:
len(data)

100

In [107]:
import random

random.shuffle(data)
train_data = data[:80]
test_data = data[80:]

In [108]:
print(len(train_data))
print(len(test_data))

80
20


In [109]:
train_data[73]

({'Computermodell': 0.05555555555555555,
  'mathematisch': 0.022222222222222223,
  'Modell': 0.022222222222222223,
  'Komplexität': 0.011111111111111112,
  'hoch': 0.011111111111111112,
  'Anzahl': 0.011111111111111112,
  'Freiheitsgrad': 0.011111111111111112,
  'Computer': 0.011111111111111112,
  'auswerten': 0.011111111111111112,
  'Auswertung': 0.03333333333333333,
  'bedeuten': 0.011111111111111112,
  'dabei': 0.011111111111111112,
  'meist': 0.011111111111111112,
  'Berechnung': 0.022222222222222223,
  'näherungsweisen': 0.011111111111111112,
  'Lösung': 0.011111111111111112,
  'numerisch': 0.011111111111111112,
  'Verfahren': 0.011111111111111112,
  'Computersimulation': 0.011111111111111112,
  'grafisch': 0.022222222222222223,
  'Darstellung': 0.022222222222222223,
  'Visualisierung': 0.022222222222222223,
  'komplex': 0.022222222222222223,
  'System': 0.011111111111111112,
  'handeln': 0.011111111111111112,
  'Z': 0.03333333333333333,
  'Architektur': 0.022222222222222223,
  'v

Wir schauen uns jetzt mal ein Dokument an:

In [110]:
train_data[27]

({'Bericht': 0.06511627906976744,
  'Report': 0.004651162790697674,
  'Informatik': 0.004651162790697674,
  'Computerprogramm': 0.009302325581395349,
  'erzeugen': 0.009302325581395349,
  'einheitlich': 0.004651162790697674,
  'strukturiert': 0.004651162790697674,
  'Darstellung': 0.004651162790697674,
  'Inhalt': 0.009302325581395349,
  'Datumenbestand': 0.004651162790697674,
  'beliebig': 0.004651162790697674,
  'elektronisch': 0.004651162790697674,
  'Form': 0.004651162790697674,
  'Bildschirminhalt': 0.004651162790697674,
  'pdf': 0.004651162790697674,
  'vorliegen': 0.004651162790697674,
  'oft': 0.009302325581395349,
  'Liste': 0.009302325581395349,
  'nennen': 0.004651162790697674,
  'weit': 0.009302325581395349,
  'Synonym': 0.009302325581395349,
  'sehen': 0.023255813953488372,
  'Teilmenge': 0.004651162790697674,
  'gespeichert': 0.004651162790697674,
  'Datum': 0.018604651162790697,
  'Beispiel': 0.023255813953488372,
  'Privatkunde': 0.004651162790697674,
  'bestehen': 0.00

Wir haben jetzt für jedes Dokument einene Merkmalsvektor. Man merke: der Vektor ist eigentlich so lang wie die Anzahl der unterschiedlichen Wörter in der ganzen Sammlung. Alle Wörter, die nicht erwähnt werden haben den Wert 0. 

Wörter, die nur in ein oder zwei Dokumementen vorkommen sind, für die Klassifikation nicht besonders nützlich. 
Wir nutzen nachher nur die Wörter, die in mindestens 5 Dokumente vorkommen. Um das vorzubereiten, berechnen wir für alle Wörter die Dokumentfrequenz>

In [111]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

In [112]:
docfreq.most_common(10)

[('einzelnachweise', 59),
 ('weblinks', 46),
 ('Literatur', 34),
 ('sehen', 33),
 ('Beispiel', 32),
 ('neu', 32),
 ('bezeichnen', 30),
 ('weit', 30),
 ('geben', 29),
 ('Entwicklung', 29)]

## Klassifikation mit Scikit Learn

Die Bibliothek Scikit Learn stellt verschiedene Klassifikationsmodelle zur Verfügung. Wir müssen jetzt richtige Merkmalsvektoren aufbauen, bei denen die Position die Bedeutung einer Zahl bestimmt. Dazu machen wir erst eine feste Liste mit allen Wörtern. Wörter, die zu selten sind lassen wir weg.

In [113]:
from sklearn import linear_model, datasets


allfeatures = [w for w in docfreq if docfreq[w] > 4]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

Wir schauen uns mal einen Vektor an:

In [114]:
train_vec[77]

[0.01559792027729636,
 0.0,
 0.0034662045060658577,
 0.0,
 0.0017331022530329288,
 0.0,
 0.0,
 0.006932409012131715,
 0.0,
 0.0,
 0.0,
 0.0017331022530329288,
 0.0,
 0.010398613518197574,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.008665511265164644,
 0.0,
 0.005199306759098787,
 0.0,
 0.0017331022530329288,
 0.005199306759098787,
 0.0034662045060658577,
 0.0,
 0.0,
 0.0017331022530329288,
 0.0,
 0.0,
 0.0,
 0.0017331022530329288,
 0.0017331022530329288,
 0.0017331022530329288,
 0.0034662045060658577,
 0.0,
 0.0,
 0.0017331022530329288,
 0.0,
 0.0,
 0.0,
 0.0017331022530329288,
 0.0,
 0.0,
 0.0034662045060658577,
 0.0,
 0.0,
 0.0,
 0.0034662045060658577,
 0.0,
 0.0017331022530329288,
 0.0017331022530329288,
 0.0017331022530329288,
 0.0,
 0.0,
 0.0017331022530329288,
 0.0017331022530329288,
 0.0,
 0.0034662045060658577,
 0.0017331022530329288,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0017331022530329288,
 0.0,
 0.0,
 0.0034662045060658577,
 0.0,
 0.005199306759098787,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,


In [115]:
train_label[77]

'Economics'

Ein einfaches, aber effektives Klassifikationsmodell, das meistens gute Ergebnisse liefert, ist die logistische Regression.

In [116]:
logreg = linear_model.LogisticRegression(C=1e9,verbose=True)
logreg.fit(train_vec,train_label)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


LogisticRegression(C=1000000000.0, verbose=True)

Wir können jetzt mit dem trainierten Classifier ein neues Dokument klassifizieren:

In [124]:
v = make_feat_vec(test_data[17][0],allfeatures) 
logreg.predict([v])

array(['Economics'], dtype='<U9')

Richtig ist:

In [125]:
test_data[17][1]

'Economics'

Oder direkt für alles Testdaten:

In [126]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

In [127]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

95.0 Prozent korrekt


Wo sind aber die Fehler aufgetreten? Un diese Frage zu beantworten, ist sogenannte eine _Konfusionsmatrix_ sehr hilfreich. NLTK hat eine Funktion zum erstellen eines Konfusionsmatrizen:

In [128]:
cm = nltk.ConfusionMatrix(test_label, pred_label)
print(cm)

          |     E |
          |     c |
          |  A  o |
          |  p  n |
          |  p  o |
          |  l  m |
          |  i  i |
          |  e  c |
          |  d  s |
----------+-------+
  Applied |<13> . |
Economics |  1 <6>|
----------+-------+
(row = reference; col = test)



Es gibt verschiedene andere Klassifikatorn, die wie nutzen können:

In [129]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_vec,train_label)
pred_label = list(knn.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

75.0 Prozent korrekt


In [130]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(train_vec,train_label)
pred_label = list(rf.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

85.0 Prozent korrekt
